In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
import random
URL = config.jwScrapeTarget["bblsite"]


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep

DRIVER_PATH = r"C:\Users\Ameno\Desktop\Drivers\chromedriver.exe"

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

s = Service(DRIVER_PATH)

driver = webdriver.Chrome(service=s)#, options=options)
driver.get(URL)
sleep(5)

In [ ]:
from functools import reduce

def dumpTextChapters(driver, chapters, out_file):
    with open(out_file,'a', encoding="UTF-8") as f:
        # f.write(f"--page: {page}--\n")
        for lnum, link in enumerate(chapters[:]):
            print("I'm opening ", link)
            driver.get(link)
            sleep(random.uniform(1,2.5))

            f.write(f"--chapter: {lnum+1}--\n")

            for passage in driver.find_elements_by_xpath(config.jwScrapeTarget["passages"]):
                passage_id = passage.get_attribute("id")
                passage_text = passage.get_attribute("innerText")
                passage_text_lines = passage_text.split("\n")

                if any(i.isdigit() for i in passage_text_lines[0]):
                    print("Skipping ", passage_text_lines[0])
                    passage_text_without_number = "\n".join(passage_text_lines[1:])
                else:
                    passage_text_without_number = "\n".join(passage_text_lines) 

                f.write(f"--passage_id: {passage_id}--\n")
                f.write(f"--passage--\n")
                f.write(passage_text_without_number)
                f.write("\n")

def getLinks(grid_name, append_id=False):
    links = []
    for rnum, row in enumerate(driver.find_elements_by_xpath(f".//ul[@class='{grid_name}']/li")):
        link_element = row.find_element_by_xpath(".//a")
        link = link_element.get_attribute("href")
        if append_id:
            id = link_element.get_attribute(config.jwScrapeTarget["bkid"])

        if link:
            if append_id:
                links.append((link, id))
            else:
                links.append(link)
    return links

def get_link_for_language(link, language):
    language_repls = config.jwScrapeTarget["languagerepls"]
    
    language_link = link
    if language != "pap":
        language_link = reduce(lambda a, kv: a.replace(*kv), language_repls[language], link)

    return language_link

In [ ]:
hebrew_grid = config.jwScrapeTarget["hbgrid"]
greek_grid = config.jwScrapeTarget["gkgrid"]

hebrew = getLinks(hebrew_grid, append_id=True)
greeks = getLinks(greek_grid, append_id=True)

bible_links = hebrew + greeks

languages = ["pap", "en", "nl"]
for link, id in bible_links[:]:
    
    for language in languages:
        language_link = get_link_for_language(link, language)
        driver.get(language_link)
        sleep(1)

        chapters_grid = config.jwScrapeTarget["chpgrid"]
        chapters = getLinks(chapters_grid)
        
        bible_out_file = config.jwScrapeTarget["outbblpre"] + f"{language}/book-{id}.txt"
        dumpTextChapters(driver, chapters, bible_out_file)

        sleep(1)
